![imagenes](logo.png)

In [ ]:
import pandas as pd
import numpy as np
from siuba import *
from siuba.dply.vector import *
from plotnine import *
import os

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
vuelos = pd.read_csv("C:\\Users\\Usuario\\Documents\\scidata\\24_cdd_py\\practicas\\aerolineas\\flights.csv")
clima = pd.read_csv("C:\\Users\\Usuario\\Documents\\scidata\\24_cdd_py\\practicas\\aerolineas\\weather.csv")
gapminder = pd.read_csv("C:\\Users\\Usuario\\Documents\\scidata\\24_cdd_py\\practicas\\gapminder\\gapminder.csv")

covid_20 = pd.read_csv("C:\\Users\\Usuario\\Documents\\scidata\\24_cdd_py\\practicas\\covid_19\\covid_2020.csv")
covid_21 = pd.read_csv("C:\\Users\\Usuario\\Documents\\scidata\\24_cdd_py\\practicas\\covid_19\\covid_2021.csv")
covid_22 = pd.read_csv("C:\\Users\\Usuario\\Documents\\scidata\\24_cdd_py\\practicas\\covid_19\\covid_2022.csv")
covid_23 = pd.read_csv("C:\\Users\\Usuario\\Documents\\scidata\\24_cdd_py\\practicas\\covid_19\\covid_2023.csv")

## Ejercicio 1.

Usando los datos de vuelos (flights), agrupa por mes y calcula el número total de vuelos retrasados. Crea un gráfico de línea que muestre la tendencia de los vuelos retrasados por mes.

In [ ]:
vuelos >> head()

In [ ]:
(
vuelos >> filter(_.dep_delay > 0) >> group_by(_.month) >> summarize(tot_retrasos = n(_)) >>
    ggplot() + geom_line(mapping=aes(x="month",y="tot_retrasos")) +
    scale_x_continuous(breaks=range(1, 13))
)

In [ ]:
(
vuelos >> filter(_.dep_delay > 0) >> group_by(_.month,_.origin) >> summarize(tot_retrasos = n(_)) >>
    ggplot() + geom_line(mapping=aes(x="month",y="tot_retrasos",color="origin")) +
    scale_x_continuous(breaks=range(1, 13))
)

## Ejercicio 2

Filtra los datos para el aeropuerto de JFK y crea un gráfico de línea que muestre la temperatura máxima diaria (temp) a lo largo del tiempo.

In [ ]:
maximas_temperaturas = (
clima >> filter(_.origin=="EWR") >> 
    group_by(_.month,_.day) >> 
    summarize(max_temp = _.temp.max()) >>
    mutate(year = 2013))

maximas_temperaturas["fecha"] = pd.to_datetime(maximas_temperaturas[["year","month","day"]])

(
maximas_temperaturas >> ggplot() + 
    geom_line(mapping=aes(x="fecha",y="max_temp",color="factor(month)")) +
    theme(axis_text_x=element_text(angle=90, hjust=0))
)

## Ejercicio 3

Usa los datos de Gapminder para comparar la evolución de la población de México y Estados Unidos a lo largo del tiempo.

In [ ]:
gapminder >> head()

In [ ]:
(
gapminder >> select(_.year,_.country,_.population) >> filter(_.country.isin(["United States","Mexico"])) >> 
    ggplot() + 
    geom_line(mapping=aes(x="year",y="population",color="country")) +
    geom_point(mapping=aes(x="year",y="population",color="country"))
)

In [ ]:
gapminder >> select(_.country) >> distinct()

## Ejercicio 4

Usa los datos de Gapminder para comparar la evolución del PIB per cápita de Japón y China.

In [ ]:
gapminder >> head()

In [ ]:
(
gapminder >> select(_.year,_.country,_.gdpPercap) >> filter(_.country.isin(["Japan","China"])) >> 
    ggplot() + 
    geom_line(mapping=aes(x="year",y="gdpPercap",color="country")) +
    geom_point(mapping=aes(x="year",y="gdpPercap",color="country"))
)

In [ ]:
americas_gdp = (gapminder >> 
    filter(_.continent == "Americas") >> 
    group_by(_.year) >> 
    summarize(gdpPercap = _.gdpPercap.mean()) >>
    mutate(country="Americas")
)

americas_gdp

In [ ]:
chi_jap_gdp = gapminder >> select(_.year,_.gdpPercap,_.country,) >> filter(_.country.isin(["Japan","China"]))

gdp = pd.concat([chi_jap_gdp,americas_gdp])

(
ggplot(data=gdp,mapping=aes(x="year",y="gdpPercap",color="country")) +
    geom_point() +
    geom_line() 
)

## Ejercicio 5

Usa los datos de vuelos (flights) para agrupar por mes y calcular el número de vuelos cancelados (is.na(dep_time)). Crea un gráfico de línea mostrando la tendencia de vuelos cancelados por mes.

## Ejercicio 6

Usa los datos meteorológicos para calcular la temperatura promedio por mes y año en el aeropuerto de LGA y crea un gráfico de línea.

## Ejercicio 7

Presenta las gráficas de contagios diarios de covid; defunciones diarias de covid y defunciones acumuladas diarias de covid.

**Extra**

- Ilumina por año

In [ ]:
covid = pd.concat([covid_20,covid_21,covid_22,covid_23])

In [ ]:
covid = covid >> mutate(año = _.FECHA_SINTOMAS.str.slice(0,4),
                mes = _.FECHA_SINTOMAS.str.slice(5,7),
                dia = _.FECHA_SINTOMAS.str.slice(8,10))

In [ ]:
(covid >> arrange(_.año,_.mes,_.dia)).tail()

In [ ]:
covid_confirmados = covid >> filter(_.CLASIFICACION_FINAL.isin([1,2,3]))

In [ ]:
## convirtamos a fecha
covid_confirmados = covid_confirmados >> mutate(fecha_sintomas = pd.to_datetime(covid_confirmados["FECHA_SINTOMAS"],dayfirst=True))

In [ ]:
conteo_confirmados = (covid_confirmados >> 
                          group_by(_.fecha_sintomas) >> 
                          summarize(total = n(_)) >>
                          mutate(año = _.fecha_sintomas.astype(str).str.slice(0,4))
                     )
conteo_confirmados


In [ ]:
(
ggplot(data=conteo_confirmados) + 
    geom_line(mapping=aes(x="fecha_sintomas",y="total",color="año")) +
    theme(axis_text_x=element_text(angle=90, hjust=0))
)

In [ ]:
covid_confirmados >> head()

In [ ]:
covid_def = covid_confirmados >> filter(_.FECHA_DEF != "9999-99-99")
covid_def = covid_def >> mutate(fecha_def = pd.to_datetime(covid_def["FECHA_DEF"],dayfirst=True))
    


In [ ]:
(covid_def >> group_by(_.fecha_def) >> summarize(total = n(_)) >> mutate(año = _.fecha_def.astype(str).str.slice(0,4)) >>
    ggplot() +
    geom_line(mapping=aes(x="fecha_def",y="total",color="año"))  +
    theme(axis_text_x=element_text(angle=90, hjust=0))
)

In [ ]:
(covid_def >> group_by(_.fecha_def) >> 
    summarize(total = n(_)) >> 
    mutate(total_acum = np.nancumsum(_.total),
           año = _.fecha_def.astype(str).str.slice(0,4)) >>
    ggplot() +
    geom_line(mapping=aes(x="fecha_def",y="total_acum",color="año"))  +
    theme(axis_text_x=element_text(angle=90, hjust=0))
)